In [ ]:
import requests
import pandas as pd

In [ ]:
amap_key = '216f07d654852385a55a1b69a537760a'

# poi_classes = '150200'
poi_classes = '150000|200000'
citys = ['湘潭县', '岳塘区', '雨湖区', '韶山市', '湘乡市']
# citys = ['湘潭县']

In [ ]:
# https://restapi.amap.com/v3/place/text?keywords=北京大学&city=beijing&output=xml&offset=20&page=1&key=<用户的key>&extensions=all
def get_pois_page(city, page):
    print('城市：' + city + '，页数：' + str(page))
    req_url = 'https://restapi.amap.com/v3/place/text?key={0}&types={1}&city={2}&output=json&offset=20&page={3}'.format(amap_key, poi_classes, city, page)
    return requests.get(req_url).json()

In [ ]:
def get_pois(city):
    i = 1
    poi_list = []
    while True:
        req_data = get_pois_page(city, i)
        if req_data['count'] == '0':
            print('城市：' + city + ' POI 获取结束。')
            break
        poi_list.extend(req_data['pois'])
        i = i + 1
    return poi_list



In [ ]:
result = []
for city in citys:
    result.extend(get_pois(city))
    

In [ ]:
## 导出

In [ ]:
df = pd.DataFrame(result, columns=['id', 'name', 'type', 'typecode', 'address', 'location', 'tel', 'pname', 'cityname', 'adname'])
# 计算经纬度
df['longitude']=df[['location']].apply(lambda x:x['location'].split(',')[0], axis=1)
df['latitude']=df[['location']].apply(lambda x:x['location'].split(',')[1], axis=1)

# 小程序数据库导入时可以识别为geopoint
df['geopoint']=df[['location']].apply(lambda x:{"type":"Point","coordinates": x['location'].split(',')}, axis=1)
df['_openid']='ouHzz5J7ED0HqOnRc80CMXOtfLqQ'

# df.to_csv('./data/test.csv', index=False, encoding='utf_8_sig')
df.to_json('./data/test.json', orient='records', lines=True, force_ascii=False) 